In [74]:

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

#warnings.filterwarnings(action='once')
import pandas as pd
from pandas import MultiIndex, Int16Dtype

import numpy as np
import math

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
# models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
import xgboost as xgb

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [75]:
#!pip3 install xgboost --upgrade

In [76]:
#df_train = pd.read_csv('../data/train_encoded_pca_cum.csv')
#df_test = pd.read_csv('../data/test_encoded_pca_cum.csv')
#df_train = pd.read_csv('../data/train_encoded_pca.csv')
#df_test = pd.read_csv('../data/test_encoded_pca.csv')

df_train = pd.read_csv('../data/train_encoded_1.csv')
df_test = pd.read_csv('../data/test_encoded_1.csv')
df_test = df_test.reindex(columns=df_train.columns.intersection(df_test.columns))

#train_encoded_pca_cum
#train_encoded_pca
#ex = pd.read_csv('../data/example-submission.csv')

In [77]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(df_train, test_size=0.2)

In [78]:
X_train = train.drop(['price', 'price_per_sqft'], axis=1)
y_train = train['price_per_sqft']

X_valid = valid.drop(['price', 'price_per_sqft'], axis=1)
y_valid = valid['price_per_sqft']

df_test_size_sqft = df_test['size_sqft'].copy()
#df_test= df_test.drop(['size_sqft'], axis=1)

np.random.seed(0)
idxs = np.arange(X_train.shape[0])
np.random.shuffle(idxs)
df_test.columns.difference(X_train.columns)


Index([], dtype='object')

In [79]:
warnings.filterwarnings('ignore')

xgb_model = xgb.XGBRegressor(eval_metric=["error"], verbose_eval=50, seed =42)

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [2, 3],
              'subsample': [0.7],
              'n_estimators': [3000],
              #'reg_alpha':[0.01],
              #'reg_lambda': [0.01],
              'num_parallel_tree':[6],
              'booster': ['gblinear'], 
              'eval_metric': [["error"]], 
              'verbose_eval': [50],
                'seed' :[42]}
#{'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 12, 'n_estimators': 1000, 'nthread': 4, 
# 'objective': 'reg:squarederror', 'subsample': 0.7}
#3217243.53476 on Kaggle

#3.
#{'booster': 'gblinear', 'colsample_bytree': 0.7, 'grow_policy': 1, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 12, 'n_estimators': 1200, 'nthread': 4, 
# 'num_parallel_tree': 3, 'objective': 'reg:squarederror', 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'subsample': 0.7}
#score -887

#3
#{'booster': 'gblinear', 'colsample_bytree': 0.7, 'grow_policy': 1, 'learning_rate': 0.05, 'max_depth': 2, 'min_child_weight': 12, 'n_estimators': 1500, 'nthread': 4,
#  'num_parallel_tree': 3, 'objective': 'reg:squarederror', 'reg_alpha': 0.05, 'reg_lambda': 0.05, 'subsample': 0.7}
# - 878

#4
#{'booster': 'gblinear', 'grow_policy': 1, 'learning_rate': 0.05, 'max_depth': 2, 
# 'n_estimators': 2500, 'nthread': 4, 'num_parallel_tree': 3, 'objective': 'reg:squarederror', 
# 'reg_alpha': 0.01, 'reg_lambda': 0.01, 'subsample': 0.7}
#score -838

#5
#{'booster': 'gblinear', 'grow_policy': 0, 'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 5000, 
# 'nthread': 4, 'num_parallel_tree': 3, 'objective': 'reg:squarederror', 'reg_alpha': 0.01, 'reg_lambda': 0.01, '
# subsample': 0.7}
#-888


clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=KFold(n_splits=2, shuffle=True, random_state=42), 
                   scoring='neg_root_mean_squared_error',
                   verbose=0, refit=True)

clf.fit(X_train, y_train)

yt_pred = clf.predict(X_train)
yv_pred = clf.predict(X_valid)

print('Training RMSE: {:.3f}'.format(mean_squared_error(yt_pred, y_train, squared=False)))
print('Test RMSE: {:.3f}'.format(mean_squared_error(yv_pred, y_valid, squared=False)))


/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr

[21:26:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "max_depth", "num_parallel_tree", "subsample", "verbose_eval" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:26:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "max_depth", "num_parallel_tree", "subsample", "verbose_eval" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:26:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "max_depth", "num_parallel_tree", "subsample", "verbose_eval" } might not be used.

  This

In [82]:
print(clf.best_params_)

print(clf.best_estimator_)

print(clf.best_score_)

print(clf.predict(df_test.copy()))


{'booster': 'gblinear', 'eval_metric': ['error'], 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 3000, 'nthread': 4, 'num_parallel_tree': 6, 'objective': 'reg:squarederror', 'seed': 42, 'subsample': 0.7, 'verbose_eval': 50}
XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None,
             eval_metric=['error'], gamma=None, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=None, max_depth=3,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=3000, n_jobs=4, nthread=4, num_parallel_tree=6,
             random_state=42, reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
             seed=42, subsample=0.7, tree_method=None, validate_parameters=1,
             verbose_eval=50, verbosity=None)
-415.3796196642709
[1731.4954 2317.2336 2310.2014 ... 2460.2104  746.9792 1609.5659]


In [83]:
yTest = clf.predict(df_test)

yTest = yTest * df_test_size_sqft
yTest = pd.DataFrame(yTest.values, columns=['Predicted'], index=df_test.index)

yTest.to_csv('../predictions/xgboost_grid_search_13.csv')

In [ ]:
#scaler = preprocessing.MinMaxScaler().fit(X_train)

# xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42, eval_metric=["error"])
# xgb_model.fit(X_train, y_train)

# # Predict labels for test data
# yt_pred = xgb_model.predict(X_train)
# yv_pred = xgb_model.predict(X_valid)

# print('Training RMSE: {:.3f}'.format(mean_squared_error(yt_pred, y_train, squared=False)))
# print('Test RMSE: {:.3f}'.format(mean_squared_error(yv_pred, y_valid, squared=False)))

# print(f'Percentage error for training data {((abs(yt_pred - y_train)/y_train)*100).mean()}')
# print(f'Percentage error for validation data {((abs(yv_pred - y_valid)/y_valid)*100).mean()}')

# xgb.plot_importance(xgb_model, height = 5, max_num_features=20)

# score = cross_val_score(xgb_model, X_train, y_train, n_jobs=-1, cv=5, scoring="r2")
# r2_mean = score.mean()

# print(f"R2 mean: {r2_mean}")
#yTest = xgb_model.predict(df_test)

# yTest = yTest * df_test_size_sqft
# yTest = pd.DataFrame(yTest.values, columns=['Predicted'], index=df_test.index)

# yTest.to_csv('../predictions/xgboost.csv')